# Monte Carlo Math

## Just math with no plotting (much faster!)

Rendering graphics is a very slow process. If we just want to estimate π, we can do so much faster with Monte Carlo integration. Here is an example using a single core: 

In [ ]:
import random
import math
import time
import numpy as np

def estimate_pi(num_points):
    points_inside_circle = 0
    for _ in range(num_points):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            points_inside_circle += 1
    return 4 * points_inside_circle / num_points

sample_sizes_sequential = np.logspace(1, 8, num=8, base=10.0, dtype=int)
time_results = {}
accuracy_results = {}
for num_samples in sample_sizes_sequential:
    start_time = time.time()
    estimated_val = estimate_pi(num_samples)
    end_time = time.time()
    elapsed_time = end_time - start_time
    time_results[num_samples] = elapsed_time
    accuracy_results[num_samples] = abs(math.pi - estimated_val)
    print(f"Estimated value with {num_samples} samples: {estimated_val} (Time taken: {elapsed_time:.4f} seconds)")

## Explanation

- Area of square = 2 * 2 = 4

- Area of circle = π * 1 * 1 = π

- Every point generated in (`random.uniform(-1, 1)` by `random.uniform(-1, 1)`) has equal probability of being anywhere inside of 4 unit area square

- Probability of falling inside the circle, then, must be:
   - $( P = \frac{\text{area of circle}}{\text{area of square}})$
   - $( P = \frac{\pi}{4})$

- Since the likelihood of points falling inside or outside the circle is effectively a direct function of their respective areas, the following is also true:

    - P = (number points inside circle / number of total points) = (π) / (4) 

    - Therefore, if we want to estimate π, we multiply P by 4

# Parallelizing the Monte Carlo π estimation

We can parallelize the Monte Carlo π estimation by splitting the total number of points into multiple chunks and estimating π for each chunk in parallel. This significantly reduces the time it takes to estimate π.

Here is an example using 128 processes:

In [ ]:
import random
import math
import time
from multiprocessing import Pool
import numpy as np

def estimate_pi(num_points):
    points_inside_circle = 0
    for _ in range(num_points):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            points_inside_circle += 1
    return points_inside_circle

def parallel_estimate_pi(num_points, num_processes):
    pool = Pool(processes=num_processes)
    points_per_process = [num_points // num_processes] * num_processes
    results = pool.map(estimate_pi, points_per_process)
    pool.close()
    pool.join()
    total_points_inside_circle = sum(results)
    return 4 * total_points_inside_circle / num_points

sample_sizes_parallel = np.logspace(1, 9, num=9, base=10.0, dtype=int)
num_processes = 128
parallel_time_results = {}
parallel_accuracy_results = {}

for num_samples in sample_sizes_parallel:
    start_time = time.time()
    estimated_val = parallel_estimate_pi(num_samples, num_processes)
    end_time = time.time()
    elapsed_time = end_time - start_time
    parallel_time_results[num_samples] = elapsed_time
    parallel_accuracy_results[num_samples] = abs(math.pi - estimated_val)
    print(f"Estimated value with {num_samples} samples: {estimated_val} (Time taken: {elapsed_time:.4f} seconds)")


# Numba

Numba is a just-in-time compiler for Python that can significantly speed up numerical computations. Here is an example using Numba:

In [ ]:
from numba import njit
import numpy as np
import time

@njit(fastmath=True)
def numba_estimate_pi(num_points):
    points_inside_circle = 0
    for _ in range(num_points):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if x**2 + y**2 <= 1:
            points_inside_circle += 1
    return 4 * points_inside_circle / num_points

# Sample sizes for testing
sample_sizes_numba = np.logspace(1, 9, num=9, base=10.0, dtype=int)
numba_time_results = {}
numba_accuracy_results = {}

for num_samples in sample_sizes_numba:
    start_time = time.time()
    estimated_val = numba_estimate_pi(num_samples)
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    # Store results using Python int as keys
    numba_time_results[int(num_samples)] = elapsed_time
    numba_accuracy_results[int(num_samples)] = abs(math.pi - estimated_val)
    print(f"Estimated value with {num_samples} samples: {estimated_val} (Time taken: {elapsed_time:.4f} seconds)")


## Analysis of Results

We can see that Numba is significantly faster than the sequential and parallel approaches.

In [ ]:
import matplotlib.pyplot as plt

# Convert all sample sizes to Python int
sample_sizes_parallel = [int(s) for s in sample_sizes_parallel]
sample_sizes_sequential = [int(s) for s in sample_sizes_sequential]  # Assuming same sizes for sequential
sample_sizes_numba = [int(s) for s in sample_sizes_numba]

# Plotting the time results
plt.figure(figsize=(14, 6))

# Time taken plot (linear scale)
plt.subplot(1, 2, 1)
plt.plot(sample_sizes_parallel, [parallel_time_results[int(s)] for s in sample_sizes_parallel], marker='o', label='Parallel Time')
plt.plot(sample_sizes_sequential, [time_results[int(s)] for s in sample_sizes_sequential], marker='o', label='Sequential Time')
plt.plot(sample_sizes_numba, [numba_time_results[int(s)] for s in sample_sizes_numba], marker='o', label='Numba Time')
plt.xlabel('Number of Samples')
plt.ylabel('Time (seconds)')
plt.title('Time taken to estimate π')
plt.legend()
plt.grid(True, linestyle='--')

# Accuracy plot (linear scale)
plt.subplot(1, 2, 2)
plt.yscale('log')
plt.plot(sample_sizes_parallel, [parallel_accuracy_results[int(s)] for s in sample_sizes_parallel], marker='o', label='Parallel Accuracy')
plt.plot(sample_sizes_sequential, [accuracy_results[int(s)] for s in sample_sizes_sequential], marker='o', label='Sequential Accuracy')
plt.plot(sample_sizes_numba, [numba_accuracy_results[int(s)] for s in sample_sizes_numba], marker='o', label='Numba Accuracy')
plt.xlabel('Number of Samples')
plt.ylabel('Error')
plt.title('Accuracy of π estimation')
plt.legend()
plt.grid(True, linestyle='--')

plt.tight_layout()
plt.show()
